# Download historical equity data for NASDAQ stocks from yahoo finance

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from time import time
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import FinanceDataReader as fdr
import yfinance as yf

In [3]:
idx = pd.IndexSlice

In [4]:
results_path = Path('KR2_results', 'asset_pricing')
if not results_path.exists():
    results_path.mkdir(parents=True)

In [5]:
def chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [6]:
def format_time(t):
    """Return a formatted time string 'HH:MM:SS
    based on a numeric time() value"""
    m, s = divmod(t, 60)
    h, m = divmod(m, 60)
    return f'{h:0>2.0f}:{m:0>2.0f}:{s:0>2.0f}'

## Get KRX symbols

In [7]:
krx = fdr.StockListing('KRX')
krx = krx[krx["Market"] != "KONEX"]
krx = krx[krx["Market"] != "KOSDAQ"]
krx['Code'] = krx['Code'] + '.' + krx['Market'].apply(lambda x: 'KS')
krx = krx["Code"].to_list()
n = len(krx)
print(f'# Symbols: {n:,.0f}')

# Symbols: 998


## Download metadata from yahoo finance

### NASDAQ symbols

In [8]:
yf_codes = yf.Tickers(krx)

In [9]:
meta_data = []
start = time()
for code in tqdm(krx):
    try:
        yf_object = yf.Ticker(code)
        s = pd.Series(yf_object.get_info())
        meta_data.append(s.to_frame(code))
    except Exception as e:
        print(code, e)

print(f'Success: {len(meta_data):5,.0f} / {len(krx):5,.0f}')

  0%|          | 0/998 [00:00<?, ?it/s]

100%|██████████| 998/998 [10:10<00:00,  1.63it/s]  

Success:   998 /   998


meta_data

In [10]:
df = pd.concat(meta_data, axis=1).dropna(how='all').T
df = df.apply(pd.to_numeric, errors='ignore')
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 005930.KS to 001525.KS
Columns: 117 entries, address1 to openInterest
dtypes: float64(89), int64(1), object(27)
memory usage: 920.0+ KB


In [11]:
results_path

WindowsPath('KR2_results/asset_pricing')

In [12]:
df.to_hdf(results_path / 'data.h5', 'stocks/info')

## Download adjusted price data using yfinance

In [13]:
prices_adj = []
start = time()
for i, chunk in enumerate(chunks(krx, 100), 1):
    prices_adj.append(yf.download(chunk, period='max', auto_adjust=True).stack(-1))

    per_ticker = (time()-start) / (i * 100)
    to_do = n - (i * 100)
    to_go = to_do * per_ticker    
    print(f'Success: {len(prices_adj):5,.0f}/{i:5,.0f} | To go: {format_time(to_go)} ({to_do:5,.0f})')

[*********************100%***********************]  100 of 100 completed


6 Failed downloads:
['247540.KS', '022100.KS', '035900.KS', '263750.KS', '066970.KS', '091990.KS']: Exception('%ticker%: No timezone found, symbol may be delisted')



Success:     1/    1 | To go: 00:00:54 (  898)


Failed to get ticker '051600.KS' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)
Failed to get ticker '267260.KS' reason: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)


[                       0%                       ]

...


9 Failed downloads:
['215200.KS', '243070.KS', '036830.KS', '091700.KS', '095610.KS', '060250.KS', '131290.KS', '084850.KS', '086450.KS']: Exception('%ticker%: No timezone found, symbol may be delisted')



Success:     4/    4 | To go: 00:01:14 (  598)
[*********************100%***********************]  100 of 100 completed


2 Failed downloads:


['230360.KS', '267980.KS']: Exception('%ticker%: No timezone found, symbol may be delisted')


Success:     5/    5 | To go: 00:00:55 (  498)
[*********************100%***********************]  100 of 100 completed
Success:     6/    6 | To go: 00:00:40 (  398)
[*********************100%***********************]  100 of 100 completed
Success:     7/    7 | To go: 00:00:27 (  298)
[*********************100%***********************]  100 of 100 completed
Success:     8/    8 | To go: 00:00:17 (  198)
[*********************100%***********************]  100 of 100 completed
Success:     9/    9 | To go: 00:00:08 (   98)
[*********************100%***********************]  98 of 98 completed
Success:    10/   10 | To go: -1:59:60 (   -2)


In [14]:
prices_adj = (pd.concat(prices_adj)
              .dropna(how='all', axis=1)
              .rename(columns=str.lower)
              .swaplevel())

In [15]:
prices_adj.index.names = ['ticker', 'date']

In [16]:
len(prices_adj.index.unique('ticker'))

924

### Remove outliers

In [17]:
df = prices_adj.close.unstack('ticker')
pmax = df.pct_change().max()
pmin = df.pct_change().min()
to_drop = pmax[pmax > 1].index.union(pmin[pmin<-1].index)
len(to_drop)

76

In [18]:
prices_adj = prices_adj.drop(to_drop, level='ticker')

In [19]:
len(prices_adj.index.unique('ticker'))

848

In [20]:
prices_adj

,,close,high,low,open,volume
ticker,date,,,,,
000100.KS,2000-01-04,3359.393799,3469.774090,3081.044028,3081.044028,728643.0
000270.KS,2000-01-04,5074.385254,5074.385254,4832.747722,4901.787017,636300.0
000720.KS,2000-01-04,283844.937500,283844.937500,258453.056019,258453.056019,70240.0
000810.KS,2000-01-04,21973.480469,22570.290206,19640.494907,19640.494907,271254.0
003490.KS,2000-01-04,10754.258789,10754.258789,9751.742671,9751.742671,2251546.0
...,...,...,...,...,...,...
36328K.KS,2023-07-24,8440.000000,8500.000000,8360.000000,8430.000000,2437.0
37550K.KS,2023-07-24,19020.000000,19370.000000,19010.000000,19370.000000,1593.0
38380K.KS,2023-07-24,9710.000000,9870.000000,9700.000000,9810.000000,2907.0


In [21]:
prices_adj.sort_index().loc[idx[:, '2000': '2023'], :].to_hdf(results_path / 'data.h5', 
                                                              'stocks/prices/adjusted')

In [22]:
prices_adj

,,close,high,low,open,volume
ticker,date,,,,,
000100.KS,2000-01-04,3359.393799,3469.774090,3081.044028,3081.044028,728643.0
000270.KS,2000-01-04,5074.385254,5074.385254,4832.747722,4901.787017,636300.0
000720.KS,2000-01-04,283844.937500,283844.937500,258453.056019,258453.056019,70240.0
000810.KS,2000-01-04,21973.480469,22570.290206,19640.494907,19640.494907,271254.0
003490.KS,2000-01-04,10754.258789,10754.258789,9751.742671,9751.742671,2251546.0
...,...,...,...,...,...,...
36328K.KS,2023-07-24,8440.000000,8500.000000,8360.000000,8430.000000,2437.0
37550K.KS,2023-07-24,19020.000000,19370.000000,19010.000000,19370.000000,1593.0
38380K.KS,2023-07-24,9710.000000,9870.000000,9700.000000,9810.000000,2907.0
